In [1]:
import time
import shutil
import requests
import threading
import multiprocessing
import concurrent

## Prints thread name and sleeps for 2 seconds

In [2]:
def sleep_fun():
    print("\nInner: starting",threading.current_thread().name)
    time.sleep(2)
    print("\nInner: ending",threading.current_thread().name)

## Runs the function

In [3]:
print("\nMain: Start")
sleep_fun()
print("\nMain: End")


Main: Start

Inner: starting MainThread

Inner: ending MainThread

Main: End


## Shows that the thread will execute independent of the main thread

In [4]:
t1 = threading.Thread(target=sleep_fun)
print("\nMain:", threading.current_thread().name)
t1.start()
print("\nMain: Finished Executing, or has it?")


Main: MainThread

Inner: starting Thread-4

Main: Finished Executing, or has it?


## Execute async using join()

In [5]:
t1 = threading.Thread(target=sleep_fun)
t1.start()
t1.join()
print("\nMain:", threading.current_thread().name)
print("\nMain: Finished Executing, yes it has!")


Inner: starting Thread-5

Inner: ending Thread-4

Inner: ending Thread-5

Main: MainThread

Main: Finished Executing, yes it has!


## Run an daemon which stops when the main thread stops (doesn't work in Jupyter as the kernal is always running)

In [6]:
# def fun_loop():
# while True:
# print("Monitoring............")
# time.sleep(1)
# t1 = threading.Thread(target=fun_loop, daemon=True)
# t1.start()
# print("\nMain: Finished Executing")

## A simple program fact(num) that runs a multiplication to time how long it takes


In [7]:
def fact(num):
    start = time.time()
    x = 1
    for i in range(1, num):
        x *= i
    time.sleep(1)
    stop = time.time() - start
    print(f'time elapsed {stop}')

In [8]:
start = time.time()
for i in range(5):
    fact(5000)
stop = time.time() - start
print(f'Total time elapsed is {stop}')

time elapsed 1.0057408809661865
time elapsed 1.004962682723999
time elapsed 1.0047054290771484
time elapsed 1.004662036895752
time elapsed 1.0045740604400635
Total time elapsed is 5.025342226028442


## Register threads

In [9]:
threads = []
num_threads = 3
for i in range(num_threads):
    print(f'registering thread {i}')
    t = threading.Thread(target=fact, args=(50000,))
    threads.append(t)


registering thread 0
registering thread 1
registering thread 2


## Start the threads and join them, so they run sync

In [10]:
start = time.time()
for t in threads:
    t.start()
for t in threads:
    t.join()
stop = time.time() - start
print(f'Total time elapsed is {stop}')


time elapsed 2.255537748336792
time elapsed 2.2595696449279785
time elapsed 2.31152606010437
Total time elapsed is 2.3121461868286133


## Using concurrent.futures to do the same thing in half the code, which runs twice as fast

In [11]:
from concurrent import futures
import time
start = time.time()
with futures.ThreadPoolExecutor() as executer:
    results = [executer.submit(fact, 5000) for i in range(3)]
stop = time.time() - start
print(f'Total time elapsed is {stop}')

time elapsed 1.004532814025879
time elapsed 1.0047590732574463
time elapsed 1.004699468612671
Total time elapsed is 1.0139708518981934


In [12]:
photos = [
    'https://unsplash.com/photos/LxaorEDmI3c/download?force=true',
    'https://unsplash.com/photos/4rDCa5hBlCs/download?force=true',
    'https://unsplash.com/photos/jFCViYFYcus/download?force=true',
    'https://unsplash.com/photos/EwKXn5CapA4/download?force=true',
    'https://unsplash.com/photos/1Z2niiBPg5A/download?force=true',
    'https://unsplash.com/photos/G15G-Any-D0/download?force=true',
    'https://unsplash.com/photos/01_igFr7hd4/download?force=true',
    'https://unsplash.com/photos/78A265wPi04/download?force=true',
    'https://unsplash.com/photos/tGTVxe0r_Rs/download?force=true',
    'https://unsplash.com/photos/hFzIoD0F_i8/download?force=true',
]

## Example of downloading images

In [13]:
response = requests.get(photos[0])
fname = response.headers['Content-Disposition'].split('filename=')[-1].replace('"', '')
print(fname)

boxed-water-is-better-LxaorEDmI3c-unsplash.jpg


In [14]:
with open('images/' + fname, 'wb') as fs:
    fs.write(response.content)


In [15]:
from pathlib import Path
def download_image(image):
    response = requests.get(image)
    if 'Content-Disposition' in response.headers:
        fname = response.headers['Content-Disposition'].split('filename=')[-1].replace('"', '')
        Path('./images').mkdir(exist_ok=True)
        fname = './images/' + fname
        with open(fname, 'wb') as fs:
            print(fname)
            fs.write(response.content)


## Using no parralelism

In [16]:
import time
import requests
start = time.time()
r = map(download_image, photos)
for i in r:
    pass
stop = time.time() - start
print(f'Total elapsed time {stop}')


./images/boxed-water-is-better-LxaorEDmI3c-unsplash.jpg
./images/casey-horner-4rDCa5hBlCs-unsplash.jpg
./images/lukasz-szmigiel-jFCViYFYcus-unsplash.jpg
./images/jeremy-bishop-EwKXn5CapA4-unsplash.jpg
./images/v2osk-1Z2niiBPg5A-unsplash.jpg
./images/boxed-water-is-better-G15G-Any-D0-unsplash.jpg
./images/qingbao-meng-01_igFr7hd4-unsplash.jpg
./images/luca-bravo-hFzIoD0F_i8-unsplash.jpg
Total elapsed time 10.94765591621399


## Using concurrent.futures

In [17]:
from concurrent.futures import ThreadPoolExecutor
start = time.time()
with ThreadPoolExecutor() as executor:
    executor.map(download_image, photos)
stop = time.time() - start
print(f'Total elapsed time {stop}')


./images/boxed-water-is-better-G15G-Any-D0-unsplash.jpg
./images/casey-horner-4rDCa5hBlCs-unsplash.jpg
./images/jeremy-bishop-EwKXn5CapA4-unsplash.jpg
./images/boxed-water-is-better-LxaorEDmI3c-unsplash.jpg
./images/lukasz-szmigiel-jFCViYFYcus-unsplash.jpg
./images/qingbao-meng-01_igFr7hd4-unsplash.jpg
./images/luca-bravo-hFzIoD0F_i8-unsplash.jpg
./images/v2osk-1Z2niiBPg5A-unsplash.jpg
Total elapsed time 1.871138572692871


## Example of locking used with threading

In [41]:
class Course:
    def __init__(self):
        self.num_student = 0

    def add_student(self, name, lock):
        print(f'Start adding student --> Thread {name}')
        with lock:
            time.sleep(1)
            my_num_student = self.num_student
            my_num_student += 1
            self.num_student = my_num_student
        print(f'Finished adding student --> Thread {name}')

In [42]:
from concurrent.futures import ThreadPoolExecutor
import threading
cs1 = Course()
lock = threading.Lock()
print(f'students: {cs1.num_student}')
with ThreadPoolExecutor(max_workers=2) as executer:
    for i in range(2):
        executer.submit(cs1.add_student, i, lock)
print(f'students: {cs1.num_student}')

students: 0
Start adding student --> Thread 0
Start adding student --> Thread 1
students: 0
